In [1]:
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg import eigsh

# Kitaev Exact diagonalization 2x2 and 3x3

firts define the operators

In [2]:
J = -1
sx = sp.csr_matrix([[0, 1], [1, 0]])
sy = sp.csr_matrix([[0, -1j], [1j, 0]])
sz = sp.csr_matrix([[1, 0], [0, -1]])
kitaev_ops = [(J * sx, sx), (J * sy, sy), (J * sz, sz)]
plaq_ops = [(sx, sy, sz, sx, sy, sz)]

then define the bonds. For the 2x2 model use

<img src="./2x2.jpeg" alt="2x2" width="600"/>

In [3]:
bonds2 = [
    ([0, 1], 0),
    ([0, 3], 1),
    ([0, 5], 2),
    ([2, 3], 0),
    ([2, 1], 1),
    ([2, 7], 2),
    ([4, 5], 0),
    ([4, 7], 1),
    ([4, 1], 2),
    ([6, 7], 0),
    ([6, 5], 1),
    ([6, 3], 2)
]

plaqs2 = [
    ([5, 0, 1, 2, 7, 6], 0),
    ([7, 2, 3, 0, 5, 4], 0),
    ([1, 4, 5, 6, 3, 2], 0),
    ([3, 6, 7, 4, 1, 0], 0)
]

And for the 3x3 model use

<img src="3x3.jpeg" alt="3x3" width=600 />

In [4]:
bonds3 = [
    ([0, 1], 0),
    ([0, 5], 1),
    ([0, 13], 2),
    ([2, 3], 0),
    ([2, 1], 1),
    ([2, 15], 2),
    ([4, 5], 0),
    ([4, 3], 1),
    ([4, 17], 2),
    ([6, 7], 0),
    ([6, 11], 1),
    ([6, 1], 2),
    ([8, 9], 0),
    ([8, 7], 1),
    ([8, 3], 2),
    ([10, 11], 0),
    ([10, 9], 1),
    ([10, 5], 2),
    ([12, 13], 0),
    ([12, 17], 1),
    ([12, 7], 2),
    ([14, 15], 0),
    ([14, 13], 1),
    ([14, 9], 2),
    ([16, 17], 0),
    ([16, 15], 1),
    ([16, 11], 2),
]

plaqs3 = [
    ([13, 0, 1, 2, 15, 14], 0),
    ([15, 2, 3, 4, 17, 16], 0),
    ([17, 4, 5, 0, 13, 12], 0),
    ([1, 6, 7, 8, 3, 2], 0),
    ([3, 8, 9, 10, 5, 4], 0),
    ([5, 10, 11, 6, 1, 0], 0),
    ([7, 12, 13, 14, 9, 8], 0),
    ([9, 14, 15, 16, 11, 10], 0),
    ([11, 16, 17, 12, 7, 6], 0)
]

Define the methods to build the sparse matrix operators 

In [5]:
def construct_operator(op_sites, operators, N):
    ret = sp.csr_matrix((2**N, 2**N), dtype=np.complex128)
    for sites, op_idx in op_sites:
        so = sorted(zip(sites, operators[op_idx]), key=lambda x: x[0], reverse=True)
        upper = N
        x = sp.identity(1)
        for site, op in so:
            lower = site
            # Non affected sites
            x = sp.kron(x, sp.identity(2**(upper-lower-1)))
            upper = lower
            # Affected sites
            x = sp.kron(x, op)
        x = sp.kron(x, sp.identity(2**upper))
        ret += x
    return ret

def eval_op(op, state):
    return state.conj().T @ (op @ state)

## 2x2 Model

In [6]:
N2 = 8
ham2 = construct_operator(bonds2, kitaev_ops, N2)
w2, v2 = eigsh(ham2)
gse2, gs2 = sorted(zip(w2, v2.T), key=lambda x: x[0])[0]
print(eval_op(ham2, gs2) / N2)

(-0.8660254037844461+0j)


In [7]:
for plaq in plaqs2:
    wp = construct_operator([plaq], plaq_ops, N2)
    print(eval_op(wp, gs2))

(-1.0000000000000087+0j)
(-1.0000000000000087+0j)
(-1.0000000000000087+0j)
(-1.0000000000000087+0j)


Energy is correct with -0.866 (see the RBM Kitaev Paper). But plaquette operator values are -1

## 3x3 Model

In [8]:
N3 = 18
ham3 = construct_operator(bonds3, kitaev_ops, N3)
w3, v3 = eigsh(ham3)
gse3, gs3 = sorted(zip(w3, v3.T), key=lambda x: x[0])[0]
print(eval_op(ham3, gs3) / N3)

(-0.7939723678960895-1.2335811384723961e-17j)


In [9]:
for plaq in plaqs3:
    wp = construct_operator([plaq], plaq_ops, N3)
    print(eval_op(wp, gs3))

(1.0000000000000386+0j)
(1.0000000000000386+8.326672684688674e-17j)
(1.000000000000039-1.3877787807814457e-17j)
(1.000000000000039+0j)
(1.0000000000000386+0j)
(1.0000000000000384+0j)
(1.0000000000000386+2.7755575615628914e-17j)
(1.0000000000000386+4.163336342344337e-17j)
(1.0000000000000384-2.7755575615628914e-17j)


Energy is correct with -0.79397 per site and plaquette expectation values are correct +1.